<a href="https://colab.research.google.com/github/alfinmaulani/skripsi/blob/master/combined(glove).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade scikit-learn
import numpy as np
import pandas as pd
import io
import os
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import gensim

     |████████████████████████████████| 23.2 MB 1.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
url0='https://raw.githubusercontent.com/alfinmaulani/skripsi/master/dataISEAR.csv'
# url1 = 'https://raw.githubusercontent.com/alfinmaulani/skripsi/master/dataisear(preputama).csv'
# url2 = 'tba'
df = pd.read_csv(url0)
# df.iloc[:,1]

In [14]:
df.emosi.value_counts()

joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1050
Name: emosi, dtype: int64

In [ ]:
df=df.drop(df[df['emosi'] == 'anger'].sample(frac=1).index)
df=df.drop(df[df['emosi'] == 'disgust'].sample(frac=1).index)

In [15]:
df.head()

,emosi,dokumen
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df2 = pd.read_csv('/content/drive/MyDrive/twitter_prosessed.csv')

preputama : lower case

In [16]:
df['dokumen']=df['dokumen'].str.lower()
# print(df.iloc[4,1])

In [17]:
df2['tweet']=df2['tweet'].str.lower()

preputama : Pembersihan noise (over enter and space)

In [18]:
df=df.replace(' \n', ' ', regex=True)
df=df.replace('   ', ' ', regex=True)
df=df.replace('  ', ' ', regex=True)

In [12]:
df2=df2.replace(' \n', ' ', regex=True)
df2=df2.replace('   ', ' ', regex=True)
df2=df2.replace('  ', ' ', regex=True)

preptambah : expand contractions

In [19]:
!pip install contractions
import contractions

     |████████████████████████████████| 321 kB 37.5 MB/s 
     |████████████████████████████████| 284 kB 39.2 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85440 sha256=edbee56a1d751a1596bbef197c553f2b8932b9b7bdada7c660044b73e2bb9393
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
df[df['dokumen'].str.contains("couldn't")]

,emosi,dokumen
124,disgust,"when i saw a picture, in the gothenburg post, ..."
526,anger,a friend suggested in a round-about way that t...
545,fear,"i was rock-climbing and absailing & felt ok, b..."
553,anger,"getting a terrible exam mark, which i couldn't..."
673,fear,"after seeing a terror movie at the cinema, i w..."
...,...,...
6589,joy,when my boyfriend came form rumphi to lilongwe...
6592,sadness,i wrote a letter to my boyfriend but he couldn...
6595,guilt,one day i found an incontinent man in the ward...
6624,joy,it was in 1983 september when mce results were...


In [20]:
expanded_words = []    

In [21]:
for word in df.dokumen:
  expanded_words.append(contractions.fix(word))

In [22]:
np.shape(expanded_words)

(7516,)

In [23]:
df.dokumen=expanded_words

In [24]:
df[df['dokumen'].str.contains("couldn't")]

,emosi,dokumen


In [25]:
expanded_words = []
for word in df2.tweet:
  expanded_words.append(contractions.fix(word))

In [26]:
df2.tweet=expanded_words

In [27]:
df2.head()

,tweet
0,is upset that he cannot update his facebook by...
1,i dived many times for the ball. managed to sa...
2,my whole body feels itchy and like its on fire
3,"no, it is not behaving at all. I am mad. why a..."
4,not the whole crew


preputama : cleansing

In [28]:
df['dokumen'] = df['dokumen'].str.replace(r'[^\w\s]+', '')

In [29]:
df.shape

(7516, 2)

In [30]:
df2['tweet'] = df2['tweet'].str.replace(r'[^\w\s]+', '')

preptambah : remove stopword

In [31]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
pat = r'\b(?:{})\b'.format('|'.join(stop))
df['dokumen'] = df['dokumen'].str.replace(pat, '')
df['dokumen'] = df['dokumen'].str.replace(r'\s+', ' ')

In [33]:
df.head()

,emosi,dokumen
0,joy,days feel close partner friends feel peace al...
1,fear,every time imagine someone love could contact ...
2,anger,obviously unjustly treated possibility elucid...
3,sadness,think short time live relate periods life thi...
4,disgust,gathering found involuntarily sitting next tw...


In [34]:
df.shape

(7516, 2)

In [35]:
df2['tweet'] = df2['tweet'].str.replace(pat, '')
df2['tweet'] = df2['tweet'].str.replace(r'\s+', ' ')

preputama : Lemmatization(d laporan salah tkoniize word)

In [36]:
sentences = [word_tokenize(x) for x in df.dokumen] 
# df = pd.DataFrame(df.apply(list,axis=1), columns=['dokumen'])

In [37]:
df.dokumen=sentences

In [38]:
sentences = [word_tokenize(x) for x in df2.tweet]

In [39]:
df2.tweet=sentences

preptambah : steming (di laporan lematization salah karna lematization = tokenize)

In [40]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

In [41]:
# temp = df.dokumen.apply(lambda x: [stemmer.stem(y) for y in x])
df['dokumen'].apply(lambda x: [stemmer.stem(y) for y in x])

0       [day, feel, close, partner, friend, feel, peac...
1       [everi, time, imagin, someon, love, could, con...
2               [obvious, unjust, treat, possibl, elucid]
3       [think, short, time, live, relat, period, life...
4       [gather, found, involuntarili, sit, next, two,...
                              ...                        
7511    [two, year, back, someon, invit, tutor, grandd...
7512    [taken, respons, someth, prepar, howev, fail, ...
7513    [home, heard, loud, sound, spit, outsid, door,...
7514          [homework, teacher, ask, us, scold, immedi]
7515    [shout, younger, brother, alway, afraid, call,...
Name: dokumen, Length: 7516, dtype: object

In [42]:
df2['tweet'].apply(lambda x: [stemmer.stem(y) for y in x])

0          [upset, can, not, updat, facebook, text, might...
1          [dive, mani, time, ball, manag, save, 50, rest...
2                     [whole, bodi, feel, itchi, like, fire]
3                             [behav, i, mad, can, not, see]
4                                              [whole, crew]
                                 ...                        
1596959                     [woke, school, best, feel, ever]
1596960     [thewdbcom, cool, hear, old, walt, interview, â]
1596961                   [readi, mojo, makeov, ask, detail]
1596962    [happi, 38th, birthday, boo, alll, time, tupac...
1596963                                              [happi]
Name: tweet, Length: 1596964, dtype: object

In [ ]:
df.head()

,emosi,dokumen
0,joy,"[days, feel, close, partner, friends, feel, pe..."
1,fear,"[every, time, imagine, someone, love, could, c..."
2,anger,"[obviously, unjustly, treated, possibility, el..."
3,sadness,"[think, short, time, live, relate, periods, li..."
4,disgust,"[gathering, found, involuntarily, sitting, nex..."


save df

In [ ]:
# df.to_csv(r'dataisear(preptambah).csv', index = False)

gabung corpus

In [54]:
sent=df2.tweet.append(df1.dokumen,ignore_index=True)

In [55]:
sent

0          [upset, can, not, update, facebook, texting, m...
1          [dived, many, times, ball, managed, save, 50, ...
2                    [whole, body, feels, itchy, like, fire]
3                          [behaving, I, mad, can, not, see]
4                                              [whole, crew]
                                 ...                        
1604475    two years back someone invited me to be the tu...
1604476    i had taken the responsibility to do something...
1604477    i was at home and i heard a loud sound of spit...
1604478    i did not do the homework that the teacher had...
1604479    i had shouted at my younger brother and he was...
Length: 1604480, dtype: object

glove

In [44]:
!pip install glove-python-binary

     |████████████████████████████████| 948 kB 27.5 MB/s 


In [45]:
from glove import Glove
from glove import Corpus

In [56]:
len(df)

7516

In [57]:
korpus = Corpus()
korpus.fit(sent, window=10, ignore_missing=False)

TypeError: ignored

In [ ]:
glove = Glove(no_components=100, learning_rate=0.05, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)
glove.fit(korpus.matrix)
glove.add_dictionary(korpus.dictionary)

cek moedel

In [ ]:
glove.word_vectors.shape

(9192, 100)

In [ ]:
glove.word_vectors[glove.dictionary['moment']]

array([-1.10971438e-03,  5.14525590e-03,  2.45005124e-03,  1.08216089e-03,
       -3.22425027e-03,  9.18803936e-04,  4.31330115e-03,  4.14382235e-03,
       -2.54231683e-03,  4.77541889e-03,  2.48021186e-03,  3.43267143e-03,
        4.91385039e-03,  2.69258277e-03, -2.87958124e-03, -9.61222870e-04,
       -1.85139329e-03,  1.80958177e-04, -9.20688453e-04, -4.75933562e-03,
       -2.99729197e-03,  1.58407096e-04,  1.67825673e-03, -5.20008366e-03,
       -5.06001142e-04,  3.25704243e-03,  1.34479741e-03,  2.39827638e-03,
       -1.02892614e-03, -2.88191580e-03,  3.78045678e-03, -3.88020930e-04,
        3.84427462e-03,  5.66356684e-03,  1.76193446e-03, -3.86543608e-03,
       -3.63495270e-04,  6.28596370e-04, -3.76530695e-03, -9.73751068e-04,
       -2.29412665e-03,  3.93164318e-03,  2.12066302e-03, -3.62973613e-03,
       -1.20828823e-03, -3.86870972e-03,  5.47193223e-04, -2.24096708e-03,
       -6.92069349e-04,  4.84144925e-03,  1.56645812e-03,  3.09914954e-03,
       -5.35436256e-03, -

ekstraksi fitur

In [ ]:
import array
ekstrak=[]
temp=array.array('i',(0,)*100)
for senten in df.dokumen:
  for word in senten:
    temp=np.add(temp,glove.word_vectors[glove.dictionary[word]])
  temp=temp/len(senten)
  ekstrak.append(temp)
has_vec=ekstrak

In [ ]:
X=has_vec
y=df.emosi
print(np.shape(X))
print(np.shape(y))
# print(X)
# print(y)

klasifikasi svm

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [ ]:
# langsung cari score

# clf = svm.SVC(kernel='poly', decision_function_shape='ovr')
# scores = cross_val_score(clf, X, y, cv=10)
# # scores.mean()

0.2851325891718843

conf matrix dan akurasi

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
clf = svm.SVC(kernel='poly', decision_function_shape='ovr')
y_pred = cross_val_predict(clf, X, y, cv=10)
ConfusionMatrixDisplay.from_predictions(y, y_pred)
plt.show()
print(accuracy_score(y, y_pred))
print(precision_recall_fscore_support(y, y_pred, average='macro'))

In [ ]:
clf = svm.SVC(kernel='rbf', decision_function_shape='ovr')
y_pred = cross_val_predict(clf, X, y, cv=10)
ConfusionMatrixDisplay.from_predictions(y, y_pred)
plt.show()
print(accuracy_score(y, y_pred))
print(precision_recall_fscore_support(y, y_pred, average='macro'))

In [ ]:
clf = svm.SVC(kernel='linear', decision_function_shape='ovr')
y_pred = cross_val_predict(clf, X, y, cv=10)
ConfusionMatrixDisplay.from_predictions(y, y_pred)
plt.show()
print(accuracy_score(y, y_pred))
print(precision_recall_fscore_support(y, y_pred, average='macro'))

In [ ]:
clf = svm.SVC(kernel='sigmoid', decision_function_shape='ovr')
y_pred = cross_val_predict(clf, X, y, cv=10)
ConfusionMatrixDisplay.from_predictions(y, y_pred)
plt.show()
print(accuracy_score(y, y_pred))
print(precision_recall_fscore_support(y, y_pred, average='macro'))